In [123]:
import pandas as pd

In [ ]:
from googleapiclient.discovery import build

api_key = 
youtube = build('youtube', 'v3', developerKey=api_key)

def get_first_channel_id(keyword):
    """Find the first YouTube channel that matches a keyword"""
    request = youtube.search().list(
        part='snippet',
        q=keyword,
        type='channel',
        maxResults=1
    )
    response = request.execute()

    if response['items']:
        return response['items']
        #return response['items'][0]['snippet']['channelId']
    else:
        return None

def get_uploads_playlist_id(channel_id):
    """Get the 'uploads' playlist ID for a channel"""
    request = youtube.channels().list(
        part='contentDetails',
        id=channel_id
    )
    response = request.execute()
    
    return response['items'][0]['contentDetails']['relatedPlaylists']['uploads']

def get_all_videos_from_playlist(playlist_id):
    """Fetch all videos from a playlist (the uploads playlist), including publish date, views, and likes"""
    videos = []
    next_page_token = None

    while True:
        request = youtube.playlistItems().list(
            part='snippet',
            playlistId=playlist_id,
            maxResults=50,
            pageToken=next_page_token
        )
        response = request.execute()

        video_ids = []
        video_snippets = {}

        for item in response['items']:
            snippet = item['snippet']
            video_id = snippet['resourceId']['videoId']
            video_ids.append(video_id)
            video_snippets[video_id] = {
                'title': snippet['title'],
                'published_at': snippet['publishedAt']
            }

        # Batch call to get video statistics
        stats_request = youtube.videos().list(
            part='statistics',
            id=','.join(video_ids)
        )
        stats_response = stats_request.execute()

        for item in stats_response['items']:
            vid = item['id']
            stats = item['statistics']
            video_info = {
                'title': video_snippets[vid]['title'],
                'video_id': vid,
                'published_at': video_snippets[vid]['published_at'],
                'views': int(stats.get('viewCount', 0)),
                'likes': int(stats.get('likeCount', 0))  # Note: This may be disabled on some videos
            }
            videos.append(video_info)

        next_page_token = response.get('nextPageToken')
        if not next_page_token:
            break

    return videos



In [125]:
res

[{'kind': 'youtube#searchResult',
  'etag': 'ETsChiW0-g7NCLeeBBhvOfLcDF0',
  'id': {'kind': 'youtube#channel', 'channelId': 'UCT1mF-u0Lcf7bYuUWTvDcpw'},
  'snippet': {'publishedAt': '2015-04-02T00:01:24Z',
   'channelId': 'UCT1mF-u0Lcf7bYuUWTvDcpw',
   'title': 'RADAL',
   'description': "The best GTA videos of all time, seriously. Streams EXCLUSIVELY on YouTube (I unlist the video after I'm done streaming ...",
   'thumbnails': {'default': {'url': 'https://yt3.ggpht.com/cVtf3ouuH6JfW1K20_yynBEfBh30j-Fwpwa_8fhUyTNSeRiKvagc6K_gZ-ZdoOepEAC0eLAD=s88-c-k-c0xffffffff-no-rj-mo'},
    'medium': {'url': 'https://yt3.ggpht.com/cVtf3ouuH6JfW1K20_yynBEfBh30j-Fwpwa_8fhUyTNSeRiKvagc6K_gZ-ZdoOepEAC0eLAD=s240-c-k-c0xffffffff-no-rj-mo'},
    'high': {'url': 'https://yt3.ggpht.com/cVtf3ouuH6JfW1K20_yynBEfBh30j-Fwpwa_8fhUyTNSeRiKvagc6K_gZ-ZdoOepEAC0eLAD=s800-c-k-c0xffffffff-no-rj-mo'}},
   'channelTitle': 'RADAL',
   'liveBroadcastContent': 'none',
   'publishTime': '2015-04-02T00:01:24Z'}}]

In [126]:
#list of all videos from a channel by keyword


# ==== Example usage ====
keyword = "radal"
res = get_first_channel_id(keyword)
channel_id=res[0]['snippet']['channelId']

if channel_id:
    print(f"✅ First matching channel ID: {channel_id}")
    playlist_id = get_uploads_playlist_id(channel_id)
    videos = get_all_videos_from_playlist(playlist_id)

    print(f"\n🎥 Found {len(videos)} videos:")
    for video in videos:
        print(f"{video['published_at']} - {video['title']} - https://youtube.com/watch?v={video['video_id']}")
else:
    print("❌ No channel found for that keyword.")


✅ First matching channel ID: UCT1mF-u0Lcf7bYuUWTvDcpw

🎥 Found 111 videos:
2025-06-17T08:01:20Z - POV: You're Unemployed. - https://youtube.com/watch?v=72yqrfumK2I
2025-06-06T20:07:09Z - He was shook 😭 - https://youtube.com/watch?v=o0GieJJ1kFE
2025-06-03T17:45:21Z - THE BIGGEST FALL OFF IN YOUTUBE HISTORY - https://youtube.com/watch?v=_sGIGPAoE04
2025-05-31T23:28:21Z - I don't know what to do - https://youtube.com/watch?v=ixcflyxplkU
2025-05-20T23:14:32Z - MILLENNIAL ENDBOSS - https://youtube.com/watch?v=-FRB1PWEEfQ
2025-05-06T19:23:17Z - RADAL REACTS TO GTA 6 TRAILER #2. - https://youtube.com/watch?v=Dy5rVWW0GaY
2025-04-26T02:44:57Z - YOU LAUGH YOU GO TO HELL - https://youtube.com/watch?v=Vwxm3eZKsWc
2025-03-30T15:09:20Z - CRINGY TIK TOK THERIANS - https://youtube.com/watch?v=Bc8SqISSAR8
2025-03-10T19:23:24Z - I made her cry with one word - https://youtube.com/watch?v=WIxVfi_DLmw
2025-03-01T19:07:31Z - I made the entire Internet hate me. (to prove how easy it is) - https://youtube.com

In [127]:
type(videos)


list

In [128]:
videos[0]

{'title': "POV: You're Unemployed.",
 'video_id': '72yqrfumK2I',
 'published_at': '2025-06-17T08:01:20Z',
 'views': 232208,
 'likes': 17709}

In [129]:
df=pd.DataFrame(videos)

In [130]:
df

,title,video_id,published_at,views,likes
0,POV: You're Unemployed.,72yqrfumK2I,2025-06-17T08:01:20Z,232208,17709
1,He was shook 😭,o0GieJJ1kFE,2025-06-06T20:07:09Z,117503,6248
2,THE BIGGEST FALL OFF IN YOUTUBE HISTORY,_sGIGPAoE04,2025-06-03T17:45:21Z,528331,29318
3,I don't know what to do,ixcflyxplkU,2025-05-31T23:28:21Z,296675,29256
4,MILLENNIAL ENDBOSS,-FRB1PWEEfQ,2025-05-20T23:14:32Z,413021,25395
...,...,...,...,...,...
106,San Andreas BATTLE ROYALE ★FUNNY MOMENTS★,vDUzX1cfCLw,2018-09-09T16:50:49Z,2286464,86265
107,San Andreas Battleroyale will end Fortnite,tujT3k0l6-A,2018-09-04T17:45:21Z,2885845,106884
108,Will this game be the new Fortnite? 🤔 (San An...,Le8eQXZXcNk,2018-08-28T14:51:03Z,2849340,94123
109,San Andreas Battleroyale better than Fortnite,fHnKhont_uE,2018-06-22T20:19:36Z,1518199,56740


In [131]:
df.dtypes

title           object
video_id        object
published_at    object
views            int64
likes            int64
dtype: object

In [132]:
df2=df.copy()

In [133]:
df2['video_id']=df2['video_id'].astype('string')

In [134]:
df2['video_id']='https://youtube.com/watch?v='+df2['video_id']

In [135]:
df2

,title,video_id,published_at,views,likes
0,POV: You're Unemployed.,https://youtube.com/watch?v=72yqrfumK2I,2025-06-17T08:01:20Z,232208,17709
1,He was shook 😭,https://youtube.com/watch?v=o0GieJJ1kFE,2025-06-06T20:07:09Z,117503,6248
2,THE BIGGEST FALL OFF IN YOUTUBE HISTORY,https://youtube.com/watch?v=_sGIGPAoE04,2025-06-03T17:45:21Z,528331,29318
3,I don't know what to do,https://youtube.com/watch?v=ixcflyxplkU,2025-05-31T23:28:21Z,296675,29256
4,MILLENNIAL ENDBOSS,https://youtube.com/watch?v=-FRB1PWEEfQ,2025-05-20T23:14:32Z,413021,25395
...,...,...,...,...,...
106,San Andreas BATTLE ROYALE ★FUNNY MOMENTS★,https://youtube.com/watch?v=vDUzX1cfCLw,2018-09-09T16:50:49Z,2286464,86265
107,San Andreas Battleroyale will end Fortnite,https://youtube.com/watch?v=tujT3k0l6-A,2018-09-04T17:45:21Z,2885845,106884
108,Will this game be the new Fortnite? 🤔 (San An...,https://youtube.com/watch?v=Le8eQXZXcNk,2018-08-28T14:51:03Z,2849340,94123
109,San Andreas Battleroyale better than Fortnite,https://youtube.com/watch?v=fHnKhont_uE,2018-06-22T20:19:36Z,1518199,56740


In [136]:
df2['title']=df2['title'].astype('string')

In [137]:
df2['published_at']=df2['published_at'].astype('string')

In [138]:
df2['published_at']=df2['published_at'].str[0:10]

In [139]:
df2

,title,video_id,published_at,views,likes
0,POV: You're Unemployed.,https://youtube.com/watch?v=72yqrfumK2I,2025-06-17,232208,17709
1,He was shook 😭,https://youtube.com/watch?v=o0GieJJ1kFE,2025-06-06,117503,6248
2,THE BIGGEST FALL OFF IN YOUTUBE HISTORY,https://youtube.com/watch?v=_sGIGPAoE04,2025-06-03,528331,29318
3,I don't know what to do,https://youtube.com/watch?v=ixcflyxplkU,2025-05-31,296675,29256
4,MILLENNIAL ENDBOSS,https://youtube.com/watch?v=-FRB1PWEEfQ,2025-05-20,413021,25395
...,...,...,...,...,...
106,San Andreas BATTLE ROYALE ★FUNNY MOMENTS★,https://youtube.com/watch?v=vDUzX1cfCLw,2018-09-09,2286464,86265
107,San Andreas Battleroyale will end Fortnite,https://youtube.com/watch?v=tujT3k0l6-A,2018-09-04,2885845,106884
108,Will this game be the new Fortnite? 🤔 (San An...,https://youtube.com/watch?v=Le8eQXZXcNk,2018-08-28,2849340,94123
109,San Andreas Battleroyale better than Fortnite,https://youtube.com/watch?v=fHnKhont_uE,2018-06-22,1518199,56740


In [140]:
df3=pd.DataFrame(df2['video_id'].copy())
df3['video_id']=df3['video_id'].str[28:]

In [141]:
df3

,video_id
0,72yqrfumK2I
1,o0GieJJ1kFE
2,_sGIGPAoE04
3,ixcflyxplkU
4,-FRB1PWEEfQ
...,...
106,vDUzX1cfCLw
107,tujT3k0l6-A
108,Le8eQXZXcNk
109,fHnKhont_uE


In [142]:
df2=df2.rename(columns={'video_id':'url'})

In [143]:
df2

,title,url,published_at,views,likes
0,POV: You're Unemployed.,https://youtube.com/watch?v=72yqrfumK2I,2025-06-17,232208,17709
1,He was shook 😭,https://youtube.com/watch?v=o0GieJJ1kFE,2025-06-06,117503,6248
2,THE BIGGEST FALL OFF IN YOUTUBE HISTORY,https://youtube.com/watch?v=_sGIGPAoE04,2025-06-03,528331,29318
3,I don't know what to do,https://youtube.com/watch?v=ixcflyxplkU,2025-05-31,296675,29256
4,MILLENNIAL ENDBOSS,https://youtube.com/watch?v=-FRB1PWEEfQ,2025-05-20,413021,25395
...,...,...,...,...,...
106,San Andreas BATTLE ROYALE ★FUNNY MOMENTS★,https://youtube.com/watch?v=vDUzX1cfCLw,2018-09-09,2286464,86265
107,San Andreas Battleroyale will end Fortnite,https://youtube.com/watch?v=tujT3k0l6-A,2018-09-04,2885845,106884
108,Will this game be the new Fortnite? 🤔 (San An...,https://youtube.com/watch?v=Le8eQXZXcNk,2018-08-28,2849340,94123
109,San Andreas Battleroyale better than Fortnite,https://youtube.com/watch?v=fHnKhont_uE,2018-06-22,1518199,56740


In [144]:
df_final=pd.concat([df2,df3],axis=1)


In [145]:
df_final

,title,url,published_at,views,likes,video_id
0,POV: You're Unemployed.,https://youtube.com/watch?v=72yqrfumK2I,2025-06-17,232208,17709,72yqrfumK2I
1,He was shook 😭,https://youtube.com/watch?v=o0GieJJ1kFE,2025-06-06,117503,6248,o0GieJJ1kFE
2,THE BIGGEST FALL OFF IN YOUTUBE HISTORY,https://youtube.com/watch?v=_sGIGPAoE04,2025-06-03,528331,29318,_sGIGPAoE04
3,I don't know what to do,https://youtube.com/watch?v=ixcflyxplkU,2025-05-31,296675,29256,ixcflyxplkU
4,MILLENNIAL ENDBOSS,https://youtube.com/watch?v=-FRB1PWEEfQ,2025-05-20,413021,25395,-FRB1PWEEfQ
...,...,...,...,...,...,...
106,San Andreas BATTLE ROYALE ★FUNNY MOMENTS★,https://youtube.com/watch?v=vDUzX1cfCLw,2018-09-09,2286464,86265,vDUzX1cfCLw
107,San Andreas Battleroyale will end Fortnite,https://youtube.com/watch?v=tujT3k0l6-A,2018-09-04,2885845,106884,tujT3k0l6-A
108,Will this game be the new Fortnite? 🤔 (San An...,https://youtube.com/watch?v=Le8eQXZXcNk,2018-08-28,2849340,94123,Le8eQXZXcNk
109,San Andreas Battleroyale better than Fortnite,https://youtube.com/watch?v=fHnKhont_uE,2018-06-22,1518199,56740,fHnKhont_uE


In [146]:
channel_id

'UCT1mF-u0Lcf7bYuUWTvDcpw'

In [147]:
res

[{'kind': 'youtube#searchResult',
  'etag': 'ETsChiW0-g7NCLeeBBhvOfLcDF0',
  'id': {'kind': 'youtube#channel', 'channelId': 'UCT1mF-u0Lcf7bYuUWTvDcpw'},
  'snippet': {'publishedAt': '2015-04-02T00:01:24Z',
   'channelId': 'UCT1mF-u0Lcf7bYuUWTvDcpw',
   'title': 'RADAL',
   'description': "The best GTA videos of all time, seriously. Streams EXCLUSIVELY on YouTube (I unlist the video after I'm done streaming ...",
   'thumbnails': {'default': {'url': 'https://yt3.ggpht.com/cVtf3ouuH6JfW1K20_yynBEfBh30j-Fwpwa_8fhUyTNSeRiKvagc6K_gZ-ZdoOepEAC0eLAD=s88-c-k-c0xffffffff-no-rj-mo'},
    'medium': {'url': 'https://yt3.ggpht.com/cVtf3ouuH6JfW1K20_yynBEfBh30j-Fwpwa_8fhUyTNSeRiKvagc6K_gZ-ZdoOepEAC0eLAD=s240-c-k-c0xffffffff-no-rj-mo'},
    'high': {'url': 'https://yt3.ggpht.com/cVtf3ouuH6JfW1K20_yynBEfBh30j-Fwpwa_8fhUyTNSeRiKvagc6K_gZ-ZdoOepEAC0eLAD=s800-c-k-c0xffffffff-no-rj-mo'}},
   'channelTitle': 'RADAL',
   'liveBroadcastContent': 'none',
   'publishTime': '2015-04-02T00:01:24Z'}}]

In [148]:
res[0]['snippet']['title']

'RADAL'

In [149]:
df_final['channel_id']=channel_id

In [150]:
df_final

,title,url,published_at,views,likes,video_id,channel_id
0,POV: You're Unemployed.,https://youtube.com/watch?v=72yqrfumK2I,2025-06-17,232208,17709,72yqrfumK2I,UCT1mF-u0Lcf7bYuUWTvDcpw
1,He was shook 😭,https://youtube.com/watch?v=o0GieJJ1kFE,2025-06-06,117503,6248,o0GieJJ1kFE,UCT1mF-u0Lcf7bYuUWTvDcpw
2,THE BIGGEST FALL OFF IN YOUTUBE HISTORY,https://youtube.com/watch?v=_sGIGPAoE04,2025-06-03,528331,29318,_sGIGPAoE04,UCT1mF-u0Lcf7bYuUWTvDcpw
3,I don't know what to do,https://youtube.com/watch?v=ixcflyxplkU,2025-05-31,296675,29256,ixcflyxplkU,UCT1mF-u0Lcf7bYuUWTvDcpw
4,MILLENNIAL ENDBOSS,https://youtube.com/watch?v=-FRB1PWEEfQ,2025-05-20,413021,25395,-FRB1PWEEfQ,UCT1mF-u0Lcf7bYuUWTvDcpw
...,...,...,...,...,...,...,...
106,San Andreas BATTLE ROYALE ★FUNNY MOMENTS★,https://youtube.com/watch?v=vDUzX1cfCLw,2018-09-09,2286464,86265,vDUzX1cfCLw,UCT1mF-u0Lcf7bYuUWTvDcpw
107,San Andreas Battleroyale will end Fortnite,https://youtube.com/watch?v=tujT3k0l6-A,2018-09-04,2885845,106884,tujT3k0l6-A,UCT1mF-u0Lcf7bYuUWTvDcpw
108,Will this game be the new Fortnite? 🤔 (San An...,https://youtube.com/watch?v=Le8eQXZXcNk,2018-08-28,2849340,94123,Le8eQXZXcNk,UCT1mF-u0Lcf7bYuUWTvDcpw
109,San Andreas Battleroyale better than Fortnite,https://youtube.com/watch?v=fHnKhont_uE,2018-06-22,1518199,56740,fHnKhont_uE,UCT1mF-u0Lcf7bYuUWTvDcpw


In [151]:
df['channel_name']=res[0]['snippet']['title']

In [152]:
df_final['channel_name']=res[0]['snippet']['title']

In [153]:
df_final

,title,url,published_at,views,likes,video_id,channel_id,channel_name
0,POV: You're Unemployed.,https://youtube.com/watch?v=72yqrfumK2I,2025-06-17,232208,17709,72yqrfumK2I,UCT1mF-u0Lcf7bYuUWTvDcpw,RADAL
1,He was shook 😭,https://youtube.com/watch?v=o0GieJJ1kFE,2025-06-06,117503,6248,o0GieJJ1kFE,UCT1mF-u0Lcf7bYuUWTvDcpw,RADAL
2,THE BIGGEST FALL OFF IN YOUTUBE HISTORY,https://youtube.com/watch?v=_sGIGPAoE04,2025-06-03,528331,29318,_sGIGPAoE04,UCT1mF-u0Lcf7bYuUWTvDcpw,RADAL
3,I don't know what to do,https://youtube.com/watch?v=ixcflyxplkU,2025-05-31,296675,29256,ixcflyxplkU,UCT1mF-u0Lcf7bYuUWTvDcpw,RADAL
4,MILLENNIAL ENDBOSS,https://youtube.com/watch?v=-FRB1PWEEfQ,2025-05-20,413021,25395,-FRB1PWEEfQ,UCT1mF-u0Lcf7bYuUWTvDcpw,RADAL
...,...,...,...,...,...,...,...,...
106,San Andreas BATTLE ROYALE ★FUNNY MOMENTS★,https://youtube.com/watch?v=vDUzX1cfCLw,2018-09-09,2286464,86265,vDUzX1cfCLw,UCT1mF-u0Lcf7bYuUWTvDcpw,RADAL
107,San Andreas Battleroyale will end Fortnite,https://youtube.com/watch?v=tujT3k0l6-A,2018-09-04,2885845,106884,tujT3k0l6-A,UCT1mF-u0Lcf7bYuUWTvDcpw,RADAL
108,Will this game be the new Fortnite? 🤔 (San An...,https://youtube.com/watch?v=Le8eQXZXcNk,2018-08-28,2849340,94123,Le8eQXZXcNk,UCT1mF-u0Lcf7bYuUWTvDcpw,RADAL
109,San Andreas Battleroyale better than Fortnite,https://youtube.com/watch?v=fHnKhont_uE,2018-06-22,1518199,56740,fHnKhont_uE,UCT1mF-u0Lcf7bYuUWTvDcpw,RADAL


In [154]:
df_final.dtypes

title           string[python]
url             string[python]
published_at    string[python]
views                    int64
likes                    int64
video_id        string[python]
channel_id              object
channel_name            object
dtype: object

In [155]:
df_final['published_at'] = pd.to_datetime(df_final['published_at'])
df_final['title'] = df_final['title'].astype(str)
df_final['url'] = df_final['url'].astype(str)
df_final['video_id'] = df_final['video_id'].astype(str)
df_final['channel_id'] = df_final['channel_id'].astype(str)
df_final['channel_name'] = df_final['channel_name'].astype(str)


In [156]:
df_final

,title,url,published_at,views,likes,video_id,channel_id,channel_name
0,POV: You're Unemployed.,https://youtube.com/watch?v=72yqrfumK2I,2025-06-17,232208,17709,72yqrfumK2I,UCT1mF-u0Lcf7bYuUWTvDcpw,RADAL
1,He was shook 😭,https://youtube.com/watch?v=o0GieJJ1kFE,2025-06-06,117503,6248,o0GieJJ1kFE,UCT1mF-u0Lcf7bYuUWTvDcpw,RADAL
2,THE BIGGEST FALL OFF IN YOUTUBE HISTORY,https://youtube.com/watch?v=_sGIGPAoE04,2025-06-03,528331,29318,_sGIGPAoE04,UCT1mF-u0Lcf7bYuUWTvDcpw,RADAL
3,I don't know what to do,https://youtube.com/watch?v=ixcflyxplkU,2025-05-31,296675,29256,ixcflyxplkU,UCT1mF-u0Lcf7bYuUWTvDcpw,RADAL
4,MILLENNIAL ENDBOSS,https://youtube.com/watch?v=-FRB1PWEEfQ,2025-05-20,413021,25395,-FRB1PWEEfQ,UCT1mF-u0Lcf7bYuUWTvDcpw,RADAL
...,...,...,...,...,...,...,...,...
106,San Andreas BATTLE ROYALE ★FUNNY MOMENTS★,https://youtube.com/watch?v=vDUzX1cfCLw,2018-09-09,2286464,86265,vDUzX1cfCLw,UCT1mF-u0Lcf7bYuUWTvDcpw,RADAL
107,San Andreas Battleroyale will end Fortnite,https://youtube.com/watch?v=tujT3k0l6-A,2018-09-04,2885845,106884,tujT3k0l6-A,UCT1mF-u0Lcf7bYuUWTvDcpw,RADAL
108,Will this game be the new Fortnite? 🤔 (San An...,https://youtube.com/watch?v=Le8eQXZXcNk,2018-08-28,2849340,94123,Le8eQXZXcNk,UCT1mF-u0Lcf7bYuUWTvDcpw,RADAL
109,San Andreas Battleroyale better than Fortnite,https://youtube.com/watch?v=fHnKhont_uE,2018-06-22,1518199,56740,fHnKhont_uE,UCT1mF-u0Lcf7bYuUWTvDcpw,RADAL


In [157]:
for column in df_final.columns:
    missing_count = df_final[column].isnull().sum()
    print(f"{column}: {missing_count} missing values")

title: 0 missing values
url: 0 missing values
published_at: 0 missing values
views: 0 missing values
likes: 0 missing values
video_id: 0 missing values
channel_id: 0 missing values
channel_name: 0 missing values


In [158]:
df_final=df_final.rename(columns={'title':'video_title'})
df_final

,video_title,url,published_at,views,likes,video_id,channel_id,channel_name
0,POV: You're Unemployed.,https://youtube.com/watch?v=72yqrfumK2I,2025-06-17,232208,17709,72yqrfumK2I,UCT1mF-u0Lcf7bYuUWTvDcpw,RADAL
1,He was shook 😭,https://youtube.com/watch?v=o0GieJJ1kFE,2025-06-06,117503,6248,o0GieJJ1kFE,UCT1mF-u0Lcf7bYuUWTvDcpw,RADAL
2,THE BIGGEST FALL OFF IN YOUTUBE HISTORY,https://youtube.com/watch?v=_sGIGPAoE04,2025-06-03,528331,29318,_sGIGPAoE04,UCT1mF-u0Lcf7bYuUWTvDcpw,RADAL
3,I don't know what to do,https://youtube.com/watch?v=ixcflyxplkU,2025-05-31,296675,29256,ixcflyxplkU,UCT1mF-u0Lcf7bYuUWTvDcpw,RADAL
4,MILLENNIAL ENDBOSS,https://youtube.com/watch?v=-FRB1PWEEfQ,2025-05-20,413021,25395,-FRB1PWEEfQ,UCT1mF-u0Lcf7bYuUWTvDcpw,RADAL
...,...,...,...,...,...,...,...,...
106,San Andreas BATTLE ROYALE ★FUNNY MOMENTS★,https://youtube.com/watch?v=vDUzX1cfCLw,2018-09-09,2286464,86265,vDUzX1cfCLw,UCT1mF-u0Lcf7bYuUWTvDcpw,RADAL
107,San Andreas Battleroyale will end Fortnite,https://youtube.com/watch?v=tujT3k0l6-A,2018-09-04,2885845,106884,tujT3k0l6-A,UCT1mF-u0Lcf7bYuUWTvDcpw,RADAL
108,Will this game be the new Fortnite? 🤔 (San An...,https://youtube.com/watch?v=Le8eQXZXcNk,2018-08-28,2849340,94123,Le8eQXZXcNk,UCT1mF-u0Lcf7bYuUWTvDcpw,RADAL
109,San Andreas Battleroyale better than Fortnite,https://youtube.com/watch?v=fHnKhont_uE,2018-06-22,1518199,56740,fHnKhont_uE,UCT1mF-u0Lcf7bYuUWTvDcpw,RADAL


In [ ]:
from sqlalchemy import create_engine
import urllib.parse

user = ''
password = urllib.parse.quote_plus('')
host = ''
port = 5432
db = ''

engine = create_engine(
    f'postgresql+psycopg2://{user}:{password}@{host}:{port}/{db}'
)

with engine.connect() as conn:
    conn.execute(text("TRUNCATE TABLE videos"))
    conn.commit()

# Then insert new data
df_final.to_sql('videos', con=engine, if_exists='append', index=False)

IntegrityError: (psycopg2.errors.ForeignKeyViolation) insert or update on table "videos" violates foreign key constraint "videos_channel_id_fkey"
DETAIL:  Key (channel_id)=(UCT1mF-u0Lcf7bYuUWTvDcpw) is not present in table "channels".

[SQL: INSERT INTO videos (video_title, url, published_at, views, likes, video_id, channel_id, channel_name) VALUES (%(video_title__0)s, %(url__0)s, %(published_at__0)s, %(views__0)s, %(likes__0)s, %(video_id__0)s, %(channel_id__0)s, %(channel_name__0)s), ( ... 16193 characters truncated ... 10)s, %(views__110)s, %(likes__110)s, %(video_id__110)s, %(channel_id__110)s, %(channel_name__110)s)]
[parameters: {'published_at__0': datetime.datetime(2025, 6, 17, 0, 0), 'url__0': 'https://youtube.com/watch?v=72yqrfumK2I', 'channel_name__0': 'RADAL', 'channel_id__0': 'UCT1mF-u0Lcf7bYuUWTvDcpw', 'video_title__0': "POV: You're Unemployed.", 'likes__0': 17709, 'views__0': 232208, 'video_id__0': '72yqrfumK2I', 'published_at__1': datetime.datetime(2025, 6, 6, 0, 0), 'ur

: 